In [2]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
import staintools
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob

In [3]:
MMRstatus_data=pd.read_csv('../../data/OriginalData/P53&MMR.csv',encoding='cp949')
def svsToTiff(imageCount):
    global svsWidth,svsHeight,path,scene,fileName
    slide_tile_size=2048
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_tile_size)):
            points = np.array([path.contains_point(np.array([widthCount* slide_tile_size, heightCount* slide_tile_size])), path.contains_point(np.array([widthCount* slide_tile_size, (heightCount+1)* slide_tile_size])), 
                                 path.contains_point(np.array([(widthCount+1)* slide_tile_size, heightCount* slide_tile_size])),
                                path.contains_point(np.array([(widthCount+1)* slide_tile_size, (heightCount+1)* slide_tile_size]))])
            inside = np.mean(points)>0
            Full=np.mean(points)>0
            if (inside):
                image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(256,256))
                if np.where(image[:,:,1]>=230)[0].shape[0]<(256*256)/4:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    TiffImageSave(image, fileName, imageCount,slide_tile_size)
                    imageCount += 1
            
    return imageCount
def TiffImageSave(image1,fileName,imageCount,slide_tile_size):
    global normalizer2x,normalizer5x,normalizer10x,tmp_img 
    cv2.imwrite('../../data/OriginalTile/'+ fileName + "_" + str(
        imageCount)+ '.tiff', image1)
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")    
    

In [4]:
svs_files=glob('../../data/OriginalData/svs/*.svs')

xml_files = [f.replace('/svs', '/xml') for f in svs_files]
xml_files = [f.replace('.svs', '.xml') for f in xml_files ]

In [94]:
for i in range(len(svs_files)):
    try:
        file_extension_index=svs_files[i].find('.svs')
        fileName=os.path.basename(os.path.splitext(svs_files[i])[0])
        slide = slideio.open_slide(svs_files[i], "SVS")
        scene = slide.get_scene(0)
        svsWidth = scene.rect[2]
        svsHeight = scene.rect[3]
        imageCount=0
        magnification=80
        tile_size=256
        xml_open=xml_files[i]
        polygon_xml = ET.parse(xml_open)
        root = polygon_xml.getroot()
        root=list(root)[0]
        root=list(root)
        imageCount=0
        for annotation_count in range(len(root)):
            root1 =list(root[0])[0]
            polygon=[]
            for polygon_count in list(root[0])[0].iter('Coordinate'): 
                polygon.append([int(float(polygon_count.attrib['X'])),int(float(polygon_count.attrib['Y']))])
            path = mpltPath.Path(polygon)
            imageCount=svsToTiff(imageCount)
    except:
        file_extension_index=svs_files[i].find('.svs')
        fileName=svs_files[i][:file_extension_index]
        print(fileName)

JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling value

KeyboardInterrupt: 